# Защита персональных данных клиентов

## Загрузка данных

In [28]:
import pandas as pd
from sklearn.datasets import make_spd_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from numpy.linalg import inv, cond
import numpy as np

In [29]:
data = pd.read_csv('/datasets/insurance.csv')

In [30]:
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Предоставленные данные состоят из 5 столбцов. 4 из них - признаки, а один - Страховые выплаты - целевой признак. Пропущенных значений нет, но можно проверить датасет на наличие дубликатов.

In [31]:
data.duplicated().value_counts()

False    4847
True      153
dtype: int64

Дубликаты есть, но так как совпадение по всем признакам маловероятно, избавимся от них.

In [32]:
data.drop_duplicates(inplace=True)

In [33]:
data.duplicated().value_counts()

False    4847
dtype: int64

Переименуем столбцы датасета для удобства.

In [34]:
data.columns = ['sex', 'age', 'salary', 'family_members', 'insurance_payments']

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?


**Ответ:** Нет, не изменится.


**Обоснование:**


Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Умножим матрицу признаков X на обратимую матрицу P:

$$
a' = (XP)w'
$$


$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке

$$
w' = (P^TX^TXP)^{-1}P^TX^Ty
$$

Используя формулу

$$ 
(A * B)^{-1} = B^{-1} * A^{-1}
$$

где A и B квадратные

$
P,P^T , X^TX квадратные
$

$$
w' = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty
$$

Умножение матрицы на обратную матрицу равно единичной матрице

$$
w' = P^{-1}(X^TX)^{-1}IX^Ty
$$

Умножение любой матрицы на единичную равно этой самой матрице.

$$
w' = P^{-1}(X^TX)^{-1}X^Ty
$$

$$
a' = XPw'
$$

$$
a' = XPP^{-1}(X^TX)^{-1}X^Ty
$$

$$
a' = X(X^TX)^{-1}X^Ty
$$

$$
a' = Xw
$$

## Алгоритм преобразования

**Алгоритм**

В качестве алгоритма преобразования примем домножение обучающих признаков X на случайную обратимую матрицу P размером 4х4. Создадим модель линейной регрессии до преобразования и посчитаем метрику R2. Далее умножим исходные признаки на обратимую матрицу и на основе полученных значений вновь посчитаем метрику R2. Метрики R2 по итогу должны быть равны.

**Обоснование**

Как было показано в предыдущем пункте, домножение признаков на какую-либо обратимую матрицу не приводит к изменению предсказаний, поэтому результаты R2 должны оказаться равны.

## Проверка алгоритма

Выделим из датасета набор признаков и целевой признак.

In [35]:
insurance_features = data.drop('insurance_payments', axis=1)

In [36]:
insurance_target = data['insurance_payments']

Получим матрицу из значений набора признаков.

In [37]:
insurance_features_matrix = insurance_features.values

In [38]:
display(insurance_features_matrix)

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

Создадим рандомную матрицу нужного размера с помощью инструмента make_spd_matrix библиотеки sklearn.

In [39]:
random_matrix = make_spd_matrix(n_dim=4, random_state=12345)
display(random_matrix)

array([[ 1.37245706, -1.03845957, -0.84389737, -0.26033015],
       [-1.03845957,  2.87886199,  1.67157893,  0.48470484],
       [-0.84389737,  1.67157893,  2.10204907,  0.3257384 ],
       [-0.26033015,  0.48470484,  0.3257384 ,  1.01695329]])

Проверим матрицу на обратимость - вероятность сгенерировать необратимую матрицу очень мала, но все же есть. При использовании необратимой матрицы дальнейшая работа по описанному алгоритму невозможна. Для проверки будем использовать функцию numpy.linalg.inv(), если матрица необратима, то код упадет с ошибкой.

In [40]:
inv(random_matrix)

array([[ 1.06341364,  0.23949912,  0.22304514,  0.08662924],
       [ 0.23949912,  0.72198481, -0.45683306, -0.13647886],
       [ 0.22304514, -0.45683306,  0.93223396, -0.0237669 ],
       [ 0.08662924, -0.13647886, -0.0237669 ,  1.07816747]])

Матрица обратима, двигаемся дальше.

In [41]:
#перемножим матрицы
transformed_matrix = insurance_features_matrix @ random_matrix
features_transformed = list(transformed_matrix)

Разделим набор признаков и целевой признак исходного датасета на обучающую и тестовую выборки.

In [42]:
features_train_origin, features_test_origin = train_test_split(insurance_features, test_size=0.25, random_state=12345)

In [43]:
target_train_origin, target_test_origin = train_test_split(insurance_target, test_size=0.25,random_state=12345)

Создадим модели.

In [44]:
model1 = LinearRegression()
model2 = LinearRegression()

Обучим первую модель, сделаем предсказание и оценим качество модели.

In [45]:
model1.fit(features_train_origin, target_train_origin)

LinearRegression()

In [46]:
predicted_origin = model1.predict(features_test_origin)
print("R2 исходного набора данных:", r2_score(target_test_origin,predicted_origin))

R2 исходного набора данных: 0.42307727492147584


Разделим набор признаков преобразованного датасета на обучающую и тестовую выборки. Обучим вторую модель, сделаем предсказание и оценим качество модели.

In [47]:
features_train_transformed, features_test_transformed = train_test_split(features_transformed, test_size=0.25, random_state=12345)

In [48]:
model2.fit(features_train_transformed, target_train_origin)

LinearRegression()

In [49]:
predicted_transformed = model2.predict(features_test_transformed)
print("R2 преобразованного набора данных:", r2_score(target_test_origin,predicted_transformed))

R2 преобразованного набора данных: 0.423077274921817


Показатели R2 очень низкие,но отличаются совсем незначительно.

Попробуем восстановить исходную матрицу, используя формулу из шага 3.

$$
Z@P^{-1} = X
$$

In [50]:
final_matrix = transformed_matrix.dot(inv(random_matrix))

In [51]:
#сравним округленные матрицы
np.round(final_matrix) == np.round(insurance_features_matrix)

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       ...,
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

# Вывод
Матрицы одинаковы. Таким образом, мы реализовали алгоритм, описанный в шаге 3 и доказали, что качество линейной регрессии не меняется при умножении матрицы признаков на любую обратимую матрицу P.